In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control.maintel.comcam import ComCam, ComCamUsages

import mt_utils

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
await asyncio.sleep(10)

In [ ]:
mtcs = MTCS(domain, intended_usage=MTCSUsages.All + MTCSUsages.Slew)
comcam = ComCam(domain, intended_usage=ComCamUsages.TakeImage)

await asyncio.gather(mtcs.start_task, comcam.start_task)

In [ ]:
mtcs.set_rem_loglevel(logging.ERROR)
mtcs.rem.mtaos.disable_check = True

In [ ]:
await mt_utils.slew_to_park(mtcs)

In [ ]:
# ack = await comcam.rem.cccamera.cmd_play.set_start(playlist="comcam-cal.playlist",
#                                                    repeat=False, timeout=60)
# print(ack)

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'

In [ ]:
await comcam.take_bias(nbias=10, group_id=group_id)

In [ ]:
await comcam.take_darks(exptime=5.0, ndarks=10, group_id=group_id)

In [ ]:
await mt_utils.slew_to_flatfield(mtcs)

In [ ]:
await comcam.take_flats(exptime=2.0, nflats=10, group_id=group_id, filter="i_06")

In [ ]:
await comcam.take_flats(exptime=2.0, nflats=10, group_id=group_id, filter="r_03")

In [ ]:
await mt_utils.slew_to_park(mtcs)